In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/hotel-booking-demand/hotel_bookings.csv")

In [ ]:
df.head()

In [ ]:
df['children'] = df['children'].fillna(0)
df['agent'] = df['agent'].astype('object').fillna('Unknown')
df['company'] = df['company'].astype('object').fillna('Unknown')

In [ ]:
df = df[df['adults'] > 0]

In [ ]:
df['arrival_date'] = pd.to_datetime(
    df['arrival_date_year'].astype(str) + "-" +
    df['arrival_date_month'].astype(str) + "-" +
    df['arrival_date_day_of_month'].astype(str)
)

df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['total_stay_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

df['total_guests'] = df['adults'] + df['children'] + df['babies']

df['room_changed'] = (df['reserved_room_type'] != df['assigned_room_type']).astype(int)

bins = [0, 3, 14, 30, 90, 365]
labels = ['0-3', '4-14', '15-30', '31-90', '90+']
df['lead_time_group'] = pd.cut(df['lead_time'], bins=bins, labels=labels)

In [ ]:
plt.figure(figsize=(6,4))
df['hotel'].value_counts().plot(kind='bar', color='skyblue')
plt.title("Bookings by Hotel Type")
plt.xlabel("Hotel Type")
plt.ylabel("Number of Bookings")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
df['arrival_date_month'].value_counts().reindex([
    'January','February','March','April','May','June','July','August',
    'September','October','November','December']).plot(kind='bar', color='lightgreen')
plt.title("Monthly Booking Distribution")
plt.xlabel("Month")
plt.ylabel("Number of Bookings")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
df['market_segment'].value_counts().plot(kind='bar', color='orange')
plt.title("Bookings by Market Segment")
plt.xlabel("Market Segment")
plt.ylabel("Number of Bookings")
plt.xticks(rotation=45)
plt.show()

In [ ]:
lead_cancel = df.groupby('lead_time_group')['is_canceled'].mean() * 100
plt.figure(figsize=(6,4))
lead_cancel.plot(marker='o', linestyle='-', color='red')
plt.title("Cancellation Rate vs Lead Time")
plt.xlabel("Lead Time Group (Days)")
plt.ylabel("Cancellation Rate (%)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
df.groupby(df['arrival_date'].dt.to_period('M'))['adr'].mean().plot(color='purple')
plt.title("Average Daily Rate (ADR) Trend Over Time")
plt.xlabel("Arrival Month")
plt.ylabel("Average ADR")
plt.grid(True)
plt.show()

In [ ]:
request_cancel = df.groupby('total_of_special_requests')['is_canceled'].mean() * 100
plt.figure(figsize=(6,4))
request_cancel.plot(marker='o', linestyle='-', color='brown')
plt.title("Cancellation Rate by Number of Special Requests")
plt.xlabel("Number of Special Requests")
plt.ylabel("Cancellation Rate (%)")
plt.grid(True)
plt.show()

***Basic Level Questions***

**Q1 What is the average lead time for bookings?**

In [ ]:
avg_lead_time = df['lead_time'].mean()
avg_lead_time

**Q2 What is the distribution of bookings by hotel type?**

In [ ]:
df['hotel'].value_counts()

**Q3 How many bookings were canceled?**

In [ ]:
total_cancellations = df['is_canceled'].sum()
total_cancellations

**Q4 What is the most common arrival month for bookings?**

In [ ]:
common_month = df['arrival_date_month'].mode()[0]
common_month

**Q5 What is the average number of special requests per booking?**

In [ ]:
avg_requests = df['total_of_special_requests'].mean()
avg_requests

**Q6 Which country has the highest number of bookings?**

In [ ]:
top_country = df['country'].value_counts().idxmax()
top_country

**Q7 What is the average daily rate (ADR) for each hotel type?**

In [ ]:
avg_adr_hotel = df.groupby('hotel')['adr'].mean()
avg_adr_hotel

**Q8 What percentage of guests required car parking spaces?**

In [ ]:
parking_percentage = (df['required_car_parking_spaces'] > 0).mean() * 100
parking_percentage

**Q9What is the average stay duration in week nights and weekend nights?**

In [ ]:
avg_week = df['stays_in_week_nights'].mean()
avg_weekend = df['stays_in_weekend_nights'].mean()
avg_week, avg_weekend

**Q10 How many bookings were made through travel agents?**

In [ ]:
bookings_through_agents = df['agent'].notna().sum()
bookings_through_agents

***Medium Level Questions***

**Q1 What is the cancellation rate for each hotel type?**

In [ ]:
cancellation_rate_hotel = df.groupby('hotel')['is_canceled'].mean() * 100
cancellation_rate_hotel

**Q2 What is the average ADR per market segment?**

In [ ]:
avg_adr_market = df.groupby('market_segment')['adr'].mean()
avg_adr_market

**Q3 What is the relationship between lead time and cancellation rate?**

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df['lead_time'], df['is_canceled'], alpha=0.2)
plt.title("Lead Time vs Cancellation Rate")
plt.xlabel("Lead Time (Days)")
plt.ylabel("Cancellation (0=No, 1=Yes)")
plt.grid(True)
plt.show()

**Q4 Which distribution channel has the highest number of bookings?**

In [ ]:
top_channel = df['distribution_channel'].value_counts()
top_channel

**Q5 What is the average number of previous cancellations by hotel type?**

In [ ]:
prev_cancel_hotel = df.groupby('hotel')['previous_cancellations'].mean()
prev_cancel_hotel

**Q6 What is the trend of ADR over the years?**

In [ ]:
plt.figure(figsize=(6,4))
df.groupby('arrival_date_year')['adr'].mean().plot(marker='o')
plt.title("ADR Trend by Year")
plt.xlabel("Year")
plt.ylabel("Average ADR")
plt.grid(True)
plt.show()

**Q7 Which month has the highest revenue?**

In [ ]:
monthly_revenue = df.groupby('arrival_date_month')['adr'].sum().sort_values(ascending=False)
monthly_revenue

**Q8 What is the impact of special requests on ADR?**

In [ ]:
special_request_adr = df.groupby('total_of_special_requests')['adr'].mean()
special_request_adr

**Q9 What is the average stay duration for repeated guests versus new guests?**

In [ ]:
df['total_stay'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
stay_comparison = df.groupby('is_repeated_guest')['total_stay'].mean()
stay_comparison

**Q10 Which room type has the highest number of bookings?**

In [ ]:
top_room_type = df['reserved_room_type'].value_counts()
top_room_type

***Advanced Level Questions***

**Q1 What factors significantly impact the cancellation rate?**

In [ ]:
import statsmodels.api as sm

features = ['lead_time', 'booking_changes', 'total_of_special_requests', 'is_repeated_guest', 'adr']
X = df[features]
y = df['is_canceled']

# Adding constant
X = sm.add_constant(X)

log_model = sm.Logit(y, X).fit()
print(log_model.summary())

**Q2 How does the ADR vary with the number of adults, children, and babies?**

In [ ]:
features = ['adults', 'children', 'babies']
X = df[features]
y = df['adr']

X = sm.add_constant(X)
reg_model = sm.OLS(y, X).fit()
print(reg_model.summary())

**Q3 What is the impact of booking changes on guest satisfaction as indicated by special requests?**

In [ ]:
corr = df['booking_changes'].corr(df['total_of_special_requests'])
corr

In [ ]:
plt.figure(figsize=(8,5))
sns.regplot(x='booking_changes', y='total_of_special_requests', data=df, scatter_kws={'alpha':0.3})

plt.title('Impact of Booking Changes on Special Requests')
plt.xlabel('Number of Booking Changes')
plt.ylabel('Total Special Requests')
plt.grid(True)
plt.show()

**Q4 What is the seasonal impact on booking cancellations?**

In [ ]:
season_cancel = df.groupby('arrival_date_month')['is_canceled'].mean() * 100
season_cancel

In [ ]:
plt.figure(figsize=(10,4))
season_cancel.plot(kind='bar', color='crimson')
plt.title("Cancellation Rate by Month")
plt.xlabel("Month")
plt.ylabel("Cancellation Rate (%)")
plt.xticks(rotation=45)
plt.show()

**Q5 How does the booking lead time distribution vary between different market segments?**

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='market_segment', y='lead_time', data=df)
plt.title("Lead Time Distribution by Market Segment")
plt.xlabel("Market Segment")
plt.ylabel("Lead Time (Days)")
plt.xticks(rotation=45)
plt.show()

***Conclusion***

The analysis of the Hotel Harmony dataset reveals critical patterns that directly influence booking demand, cancellation behavior, pricing decisions, and guest satisfaction. City Hotels attract more bookings due to location convenience, while Resort Hotels generate higher revenue per stay, especially during peak seasons. Seasonal trends show strong demand variations, emphasizing the need for dynamic pricing, seasonal promotions, and capacity planning.

Cancellation behavior is strongly driven by long lead times, booking changes, and low deposit types, highlighting that guests who book far in advance are more likely to cancel. Implementing partial payment or stricter cancellation policies for high-lead bookings can significantly reduce last-minute losses. The study also finds that the Average Daily Rate (ADR) fluctuates by season and guest composition, suggesting pricing optimization based on market segment and family-focused booking trends.

Furthermore, guests who request more special services generally show lower cancellation tendencies, indicating that customization improves commitment and satisfaction. Encouraging personalized services can therefore enhance loyalty and reduce booking churn.